## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions,Date
0,0,Albany,US,42.6001,-73.9662,39.00,65,96,4.00,overcast clouds,2021-01-17 16:29:12
1,1,Calvia,ES,39.5657,2.5062,59.00,67,0,10.36,clear sky,2021-01-17 16:29:12
2,2,Son La,VN,21.3167,103.9000,47.41,79,97,3.11,overcast clouds,2021-01-17 16:29:12
3,3,Rikitea,PF,-23.1203,-134.9692,78.03,77,0,15.26,light rain,2021-01-17 16:29:13
4,4,Lebu,CL,-37.6167,-73.6500,61.05,72,0,20.54,clear sky,2021-01-17 16:29:13


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions,Date
3,3,Rikitea,PF,-23.1203,-134.9692,78.03,77,0,15.26,light rain,2021-01-17 16:29:13
8,8,Itabaiana,BR,-10.6850,-37.4253,86.00,62,20,13.80,few clouds,2021-01-17 16:29:13
9,9,Atuona,PF,-9.8000,-139.0333,78.03,76,1,15.52,clear sky,2021-01-17 16:29:13
10,10,Vanimo,PG,-2.6741,141.3028,80.02,85,88,6.46,light rain,2021-01-17 16:29:13
13,13,Nsunga,TZ,-1.1439,31.3947,77.00,65,20,5.75,few clouds,2021-01-17 16:29:13
25,25,Aketi,CD,2.7388,23.7833,86.94,46,78,2.37,broken clouds,2021-01-17 16:29:14
27,27,Niquelandia,BR,-14.4739,-48.4597,80.91,63,100,2.62,moderate rain,2021-01-17 16:29:14
34,34,Puerto Ayora,EC,-0.7393,-90.3518,79.00,73,75,9.22,broken clouds,2021-01-17 16:25:57
35,35,Vaini,TO,-21.2000,-175.2000,78.80,94,40,4.61,scattered clouds,2021-01-17 16:29:15
40,40,Mahebourg,MU,-20.4081,57.7000,80.60,83,40,12.66,scattered clouds,2021-01-17 16:29:16


In [17]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID               224
City                  224
Country               224
Lat                   224
Lng                   224
Max Temp              224
Humidity              224
Cloudiness            224
Wind Speed            224
Current Conditions    224
Date                  224
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Conditions", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
3,Rikitea,PF,78.03,light rain,-23.1203,-134.9692,
8,Itabaiana,BR,86.00,few clouds,-10.6850,-37.4253,
9,Atuona,PF,78.03,clear sky,-9.8000,-139.0333,
10,Vanimo,PG,80.02,light rain,-2.6741,141.3028,
13,Nsunga,TZ,77.00,few clouds,-1.1439,31.3947,
25,Aketi,CD,86.94,broken clouds,2.7388,23.7833,
27,Niquelandia,BR,80.91,moderate rain,-14.4739,-48.4597,
34,Puerto Ayora,EC,79.00,broken clouds,-0.7393,-90.3518,
35,Vaini,TO,78.80,scattered clouds,-21.2000,-175.2000,
40,Mahebourg,MU,80.60,scattered clouds,-20.4081,57.7000,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df= hotel_df[hotel_df['Hotel Name']!= ""]
hotel_df.head()

,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
3,Rikitea,PF,78.03,light rain,-23.1203,-134.9692,Pension Maro'i
8,Itabaiana,BR,86.00,few clouds,-10.6850,-37.4253,Pousada Santana
9,Atuona,PF,78.03,clear sky,-9.8000,-139.0333,Villa Enata
10,Vanimo,PG,80.02,light rain,-2.6741,141.3028,Vahmoneh Lodge Vanimo
27,Niquelandia,BR,80.91,moderate rain,-14.4739,-48.4597,Hotel Suinã


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
